# Import Libraries

In [1]:
from tabula import read_pdf
import pandas as pd
import os
import re
import numpy as np

# Import data

In [2]:
raw_df = read_pdf('2016 and 2017 DOJ Prosecution Data.pdf', pages='all')

In [3]:
df = raw_df.copy()

# Clean Data

## Label Districts

In [4]:
districtRegex = re.compile('District \d+')

In [5]:
def matchDistrict(x):
    try:
        if re.match(districtRegex, x):
            return re.search(districtRegex, x).group(0)
    except:
        return np.nan

In [6]:
df['District'] = df.apply(lambda x: matchDistrict(x['District 1']), axis = 1)

## Shift around rows that should be headers to make each table the same length

Each table in the pdf is merged into a single `DataFrame`. Each table should have the same number of rows. The following code extracts the District number, removes the header row and makes the number of rows in each table consistent

In [7]:
data = [np.nan] * 4 + ['District 1']
df.loc[-1] = data
df.index = df.index + 1
df = df.sort_index()

In [8]:
df = df.dropna(how='all')

Fill with the same District name until the District name changes

In [9]:
df['District'] = df['District'].fillna(method='ffill')

In [10]:
df.columns = ['Action', 'Action2', '2016', '2017', 'District']

Remove total columns and total arrests (the data roll up into this calculation)

In [11]:
maskTotal = ((df['Action2'].str.startswith('Total', na=False)) | (df['Action'] == 'Arrests'))
df = df.loc[~maskTotal]

In [12]:
# drop first row
df = df.drop([0])

In [13]:
df.reset_index(drop = True, inplace = True)

In [14]:
df = df.loc[~((df['2016']=='2016') & (df['2017'] == '2017'))]
df.reset_index(drop=True, inplace=True)

In [15]:
def combineAction(x):
    try:
        if np.isnan(x['Action']):
            return x['Action2']
    except:
        return x['Action']

In [16]:
df['Activity'] = df.apply(lambda x: combineAction(x), axis=1)

In [17]:
df= df.drop(columns = ['Action', 'Action2'], axis = 1)
df = df[['District', 'Activity', '2016', '2017']]

Check for typos - there appears to be 7 instances where the pdf did not read perfectly (e.g. `Jury Trial Court Trial` as a single value. Split each of those instances into separate rows

In [18]:
df['Activity'].value_counts()

Pending Cases             49
Papered Cases             49
Not Guilty Verdict        49
No Papered Cases          49
Dismissals                49
Guilty Pleas              49
Court Trial               42
Jury Trial                42
Jury Trial Court Trial     7
Name: Activity, dtype: int64

Find indices of where these typos occur

In [19]:
indices = list(df[df['Activity'] == 'Jury Trial Court Trial'].index)
indices

[18, 73, 128, 183, 238, 293, 348]

In [20]:
def splitRow(i):
    new_row = df.iloc[i].copy()
    df.iloc[i]['Activity'], new_row['Activity'] = re.findall('\w+ Trial', df.iloc[i]['Activity'])
    df.iloc[i]['2016'], new_row['2016'] = df.iloc[i]['2016'].split()
    df.iloc[i]['2017'], new_row['2017'] = df.iloc[i]['2017'].split()
    
    return df.iloc[i], new_row


In [21]:
count = 0
for i in indices:
    i += count
    x, y = splitRow(i)
    df= pd.concat([df.iloc[:i+1], pd.DataFrame(y).transpose(), df.iloc[i+1:]]).reset_index(drop=True)
    count += 1
    

In [22]:
df['Activity'].value_counts()

Pending Cases         49
Papered Cases         49
Not Guilty Verdict    49
No Papered Cases      49
Dismissals            49
Court Trial           49
Guilty Pleas          49
Jury Trial            49
Name: Activity, dtype: int64

In [23]:
df['District'].value_counts()

District 7    56
District 1    56
District 5    56
District 4    56
District 6    56
District 2    56
District 3    56
Name: District, dtype: int64

In [24]:
offenses = ['Aggravated Assault',
            'Arson',
            'Burglary',
            'First Degree Sexual/Child Abuse',
            'First Degree Theft',
            'Homicide',
            'UUV']

In [25]:
df['Offense'] = np.repeat(offenses, 56)

In [26]:
df = df[['Offense', 'District', 'Activity', '2016', '2017']]


In [27]:
df.columns = ['Offense', 'District', 'Activity', '2016', '2017']

In [28]:
df.head()

,Offense,District,Activity,2016,2017
0,Aggravated Assault,District 1,No Papered Cases,2,3
1,Aggravated Assault,District 1,Papered Cases,13,22
2,Aggravated Assault,District 1,Jury Trial,0,0
3,Aggravated Assault,District 1,Court Trial,0,0
4,Aggravated Assault,District 1,Guilty Pleas,7,10


In [29]:
categoryDict = {
    'No Papered Cases': 'Arrest',
    'Papered Cases': 'Arrest',
    'Jury Trial': 'Conviction',
    'Court Trial': 'Conviction',
    'Guilty Pleas': 'Conviction',
    'Dismissals': 'No Conviction',
    'Not Guilty Verdict': 'No Conviction',
    'Pending Cases': 'In Progress'
}

In [30]:
df['Status'] = df['Activity'].apply(lambda x: categoryDict[x])

In [31]:
df = df[['Offense', 'District', 'Status', 'Activity', '2016', '2017']]

In [32]:
df.to_csv('DC_DOJ_Prosecution_Data.csv')

In [34]:
df.to_excel('DC_DOJ_Prosecution_Data.xlsx')